# TensorFlowによる多層CNNの実装

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(42)
tf.set_random_seed(42)

Instructions for updating:
Use the retry module or similar alternatives.


In [3]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
num_filters1 = 32

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1, 28, 28, 1])

W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,num_filters1], stddev = 0.1))

h_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding='SAME')

b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1])) #ReLUでカットする閾値を決める 初期値は0.1
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)

h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding = 'SAME')

In [5]:
num_filters2 = 64

W_conv2 = tf.Variable(tf.truncated_normal([5,5,num_filters1, num_filters2], stddev=0.1))

h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding='SAME')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [6]:
h_pool2_flat = tf.reshape(h_pool2,[-1, 7*7*num_filters2])

num_units1 = 7*7*num_filters2
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) +b2)

keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2_drop, w0) +b0)

In [7]:
t = tf.placeholder(tf.float32, [None, 10])
loss = - tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(var_list={"b0": b0, "w0": w0, "b2": b2, "w2": w2})

In [9]:
i = 0
for _ in range(20000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step, feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
    if i % 500 == 0:
        loss_vals, acc_vals = [], []
        for c in range(5):
            start = len(mnist.test.labels) // 5 *c
            end = len(mnist.test.labels) // 5 * (c+1)
            loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x:mnist.test.images[start:end], t:mnist.test.labels[start:end], keep_prob:1.0})
            
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print('Step: %d, Loss: %f, Accuracy: %f' % (i, loss_val, acc_val))
        saver.save(sess, './cnn_session', global_step=i)

Step: 500, Loss: 1477.868408, Accuracy: 0.954900
Step: 1000, Loss: 1003.672241, Accuracy: 0.971200
Step: 1500, Loss: 736.695618, Accuracy: 0.977000
Step: 2000, Loss: 633.848755, Accuracy: 0.979300
Step: 2500, Loss: 624.459229, Accuracy: 0.978400
Step: 3000, Loss: 538.565918, Accuracy: 0.982500
Step: 3500, Loss: 522.744812, Accuracy: 0.983600
Step: 4000, Loss: 452.990753, Accuracy: 0.984800
Step: 4500, Loss: 475.163910, Accuracy: 0.984200
Step: 5000, Loss: 468.982941, Accuracy: 0.984100
Step: 5500, Loss: 415.640778, Accuracy: 0.986300
Step: 6000, Loss: 407.362915, Accuracy: 0.986600
Step: 6500, Loss: 374.767334, Accuracy: 0.987200
Step: 7000, Loss: 445.587585, Accuracy: 0.985600
Step: 7500, Loss: 346.484314, Accuracy: 0.988100
Step: 8000, Loss: 339.608765, Accuracy: 0.988400
Step: 8500, Loss: 363.998840, Accuracy: 0.987200
Step: 9000, Loss: 354.237793, Accuracy: 0.988400
Step: 9500, Loss: 312.068390, Accuracy: 0.989900
Step: 10000, Loss: 296.044739, Accuracy: 0.991200
Step: 10500, Loss:

In [1]:
! ls cnn_session*

cnn_session-100.data-00000-of-00001   cnn_session-19000.meta
cnn_session-100.index                 cnn_session-19500.data-00000-of-00001
cnn_session-100.meta                  cnn_session-19500.index
cnn_session-150.data-00000-of-00001   cnn_session-19500.meta
cnn_session-150.index                 cnn_session-200.data-00000-of-00001
cnn_session-150.meta                  cnn_session-200.index
cnn_session-18000.data-00000-of-00001 cnn_session-200.meta
cnn_session-18000.index               cnn_session-20000.data-00000-of-00001
cnn_session-18000.meta                cnn_session-20000.index
cnn_session-18500.data-00000-of-00001 cnn_session-20000.meta
cnn_session-18500.index               cnn_session-50.data-00000-of-00001
cnn_session-18500.meta                cnn_session-50.index
cnn_session-19000.data-00000-of-00001 cnn_session-50.meta
cnn_session-19000.index
